In [1]:
import os
import pandas as pd
import numpy as np
import PIL
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from sklearn import model_selection

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

## Datos

In [3]:
path = '/home/maryskal/Downloads/Garbage classification'
path = '/home/mr1142/Documents/Data/Garabge'

In [4]:
classes = os.listdir(path)

In [5]:
directorios = []
clases = []
for clase in classes:
    dir_list = [os.path.join(path, clase, dir) for dir in os.listdir(os.path.join(path, clase))]
    directorios += dir_list
    clases += [clase for i in range(len(dir_list))]

In [6]:
df = pd.DataFrame()
df['directorios'] = directorios
df['clase'] = clases

In [7]:
df = pd.concat([df, pd.get_dummies(df.clase)], axis = 1)

In [8]:
df.head()

,directorios,clase,cardboard,glass,metal,paper,plastic,trash
0,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0
1,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0
2,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0
3,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0
4,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0


In [9]:
df = df.sample(frac=1).reset_index(drop = True)
df.head()

,directorios,clase,cardboard,glass,metal,paper,plastic,trash
0,/home/mr1142/Documents/Data/Garabge/paper/pape...,paper,0,0,0,1,0,0
1,/home/mr1142/Documents/Data/Garabge/plastic/pl...,plastic,0,0,0,0,1,0
2,/home/mr1142/Documents/Data/Garabge/cardboard/...,cardboard,1,0,0,0,0,0
3,/home/mr1142/Documents/Data/Garabge/paper/pape...,paper,0,0,0,1,0,0
4,/home/mr1142/Documents/Data/Garabge/glass/glas...,glass,0,1,0,0,0,0


In [10]:
def charge_reshape(path):
    img = Image.open(path)
    img = img.resize((256,256))
    img = np.array(img)
    try:
        img = img[:,:,0]
    except:
        print('e')
    img = np.expand_dims(img, axis=-1)
    img = img/256
    return img

In [11]:
tensor = np.zeros((len(df),256,256,1))
for i in range(len(df)):
    tensor[i,...] = charge_reshape(df.directorios[i])

Como son 6 variables de salida hay que meter como output un array que tenga 6 valores.

In [12]:
X = tensor
Y = df[df.columns[2:8]].to_numpy()

In [13]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, 
                                                                    random_state=42, shuffle=True, stratify=Y )

In [14]:
len(Y_train)

2021

In [15]:
print(list(df.columns[2:]))
print(np.sum(Y_train, axis=0)/np.sum(Y_train)*100)
print(np.sum(Y_test, axis=0)/np.sum(Y_test)*100)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
[15.93270658 19.84166254 16.22958931 23.50321623 19.04997526  5.44285007]
[16.00790514 19.76284585 16.2055336  23.51778656 19.16996047  5.33596838]


## Modelo

Cogemos el modelo efficienNet con los pesos de imagenet

### EfficientNet

In [16]:
input_shape = (256,256,3)
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape)

In [17]:
type(conv_base)

tensorflow.python.keras.engine.functional.Functional

In [18]:
len(conv_base.layers)

237

In [19]:
# conv_base.summary()

### Modelo

- Añado una capa inicial para poder entregarle el input de 3 canales. 
- Añado 1 convolución con max pooling y una segunda convolución cno globalMaxPooling para ajustar mejor
- Añado Dropout para evitar overfiting
- Añado la capa final con 6 neuronas de salida

In [20]:
model = models.Sequential()
model.add(layers.Conv2D(3,3,padding="same", input_shape=(256,256,1), activation='elu', name = 'conv_inicial'))
model.add(conv_base)
model.add(layers.Conv2D(3,32, padding='same', input_shape=(8,8,1280), activation='selu', name = 'conv_posterior'))
model.add(layers.MaxPool2D(pool_size = (2,2), padding='same', name = 'first_pooling'))
model.add(layers.Conv2D(3,64, padding='same', input_shape=(4,4,1280), activation='selu', name = 'last_convolution'))
model.add(layers.GlobalMaxPooling2D(name="general_max_pooling"))
model.add(layers.Dropout(0.2, name="dropout_out"))
model.add(layers.Dense(len(np.unique(df.clase)), activation="sigmoid", name="fc_out"))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_inicial (Conv2D)        (None, 256, 256, 3)       30        
_________________________________________________________________
efficientnetb0 (Functional)  (None, 8, 8, 1280)        4049571   
_________________________________________________________________
conv_posterior (Conv2D)      (None, 8, 8, 3)           3932163   
_________________________________________________________________
first_pooling (MaxPooling2D) (None, 4, 4, 3)           0         
_________________________________________________________________
last_convolution (Conv2D)    (None, 4, 4, 3)           36867     
_________________________________________________________________
general_max_pooling (GlobalM (None, 3)                 0         
_________________________________________________________________
dropout_out (Dropout)        (None, 3)                 0

### 1º entrenamiento
Solo vamos a entrenar las capas que hemos añadido

In [22]:
conv_base.trainable = False

In [23]:
len(model.trainable_variables)

8

Escogemos el optimizador Adam y funcion de coste CategoricalCrosentropy

In [24]:
lr = 1e-4

In [25]:
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.BinaryCrossentropy()

In [26]:
model.compile(optimizer=opt, loss = loss , metrics = ['CategoricalAccuracy', 'AUC'])

#### Entrenamiento

In [27]:
batch_size = 16
initial_epochs = 100

In [28]:
print('loss, acuracy, auc')
model.evaluate(X_test, Y_test)

loss, acuracy, auc
16/16 [==============================] - 102s 3s/step - loss: 0.6942 - categorical_accuracy: 0.2011 - auc: 0.4720


[0.6942886710166931, 0.197628453373909, 0.4669960141181946]

In [ ]:
history = model.fit(X_train,Y_train, batch_size = batch_size, epochs = initial_epochs, 
                    validation_data = (X_test, Y_test))

Epoch 1/100
127/127 [==============================] - 260s 1s/step - loss: 0.6598 - categorical_accuracy: 0.2053 - auc: 0.5228 - val_loss: 0.6298 - val_categorical_accuracy: 0.1976 - val_auc: 0.5518
Epoch 2/100
127/127 [==============================] - 18s 140ms/step - loss: 0.6374 - categorical_accuracy: 0.2019 - auc: 0.5377 - val_loss: 0.6130 - val_categorical_accuracy: 0.1976 - val_auc: 0.4941
Epoch 3/100
127/127 [==============================] - 18s 141ms/step - loss: 0.6244 - categorical_accuracy: 0.1900 - auc: 0.5397 - val_loss: 0.5926 - val_categorical_accuracy: 0.1976 - val_auc: 0.5668
Epoch 4/100
127/127 [==============================] - 18s 141ms/step - loss: 0.6110 - categorical_accuracy: 0.1974 - auc: 0.5361 - val_loss: 0.5770 - val_categorical_accuracy: 0.1976 - val_auc: 0.5561
Epoch 5/100
127/127 [==============================] - 18s 141ms/step - loss: 0.5991 - categorical_accuracy: 0.2014 - auc: 0.5334 - val_loss: 0.5627 - val_categorical_accuracy: 0.1976 - val_auc:

127/127 [==============================] - 19s 146ms/step - loss: 0.4691 - categorical_accuracy: 0.1999 - auc: 0.5595 - val_loss: 0.4436 - val_categorical_accuracy: 0.1976 - val_auc: 0.5972
Epoch 42/100
127/127 [==============================] - 18s 145ms/step - loss: 0.4662 - categorical_accuracy: 0.1964 - auc: 0.5629 - val_loss: 0.4416 - val_categorical_accuracy: 0.2352 - val_auc: 0.6049
Epoch 43/100
127/127 [==============================] - 18s 145ms/step - loss: 0.4674 - categorical_accuracy: 0.1910 - auc: 0.5591 - val_loss: 0.4411 - val_categorical_accuracy: 0.1976 - val_auc: 0.5976
Epoch 44/100
127/127 [==============================] - 18s 143ms/step - loss: 0.4650 - categorical_accuracy: 0.1841 - auc: 0.5636 - val_loss: 0.4405 - val_categorical_accuracy: 0.1976 - val_auc: 0.5972
Epoch 45/100
127/127 [==============================] - 19s 146ms/step - loss: 0.4672 - categorical_accuracy: 0.1885 - auc: 0.5544 - val_loss: 0.4398 - val_categorical_accuracy: 0.1976 - val_auc: 0.597

In [ ]:
import re
len(history.history.keys())/2

In [ ]:
epochs_range = range(initial_epochs)
i = 0
for i in range(int(len(history.history.keys())/2)):
    key = list(history.history.keys())[i]
    
    plt.figure(figsize=(5, 5))
    plt.subplot(1, 1, 1)
    plt.plot(epochs_range, history.history[key], label=key)
    plt.plot(epochs_range, history.history['val_' + key], label='val_' + key)
    plt.legend(loc='lower right')
    plt.title('Training and Validation')
plt.show()

### 2º Entrenamiento
Ahora vamos a entrenar tambien la red preentrenada, pero no queremos entrenar todas las capas, únicamente las capas más profundas, ya que las primeras capas son muy genéricas.

In [ ]:
conv_base.trainable = True

In [ ]:
len(conv_base.layers)

Vamos a entrenar desde la capa 100 en adelante

In [ ]:
fine_tune_at = 100

for layer in conv_base.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
len(model.trainable_variables)

Como el modelo es mucho más grande tenemos que usar un learning rate más bajo del que usabamos.
Vamos a utilizar dos learning rates, uno para el modelo preentrenado y otro para las capas nuevas

In [ ]:
opt = [tf.keras.optimizers.Adam(learning_rate = lr), tf.keras.optimizers.RMSprop(learning_rate = lr/100)]
optimizers_and_layers = [(opt[0], model.layers[0]),
                         (opt[1], model.layers[1]),
                         (opt[0], model.layers[2:])]
opt = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
model.compile(optimizer=opt, loss = loss , metrics = ['CategoricalAccuracy', 'AUC'])

#### Entrenamiento

In [ ]:
batch_size = 60
fine_tune_epochs = 20
total_epoch = initial_epochs + fine_tune_epochs

In [ ]:
history = model.fit(X_train,Y_train, 
                    batch_size = batch_size,
                    epochs = total_epoch,
                    initial_epoch=history.epoch[-1],
                    validation_data = (X_test, Y_test))

In [ ]:
epochs_range = len(history.history['loss'])
i = 0
for i in range(int(len(history.history.keys())/2)):
    key = list(history.history.keys())[i]
    
    plt.figure(figsize=(5, 5))
    plt.subplot(1, 1, 1)
    plt.plot(history.history[key], label=key)
    plt.plot(history.history['val_' + key], label='val_' + key)
    plt.legend(loc='lower right')
    plt.title('Training and Validation')
plt.show()